In [ ]:
!apt-get update -qq
!apt-get install -y aria2
!pip install -q msal requests tqdm
!curl https://rclone.org/install.sh | bash

# Movie Upload from URL


In [ ]:
import os
from urllib.parse import urlparse, unquote
import subprocess

DOWNLOAD_URL = input("Enter Download URL: ")

def get_filename_from_aria2(url):
    try:
        result = subprocess.run(
            ['aria2c', '--dry-run', url],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            check=True
        )

        dir = os.getcwd().replace("\\", "/")
        fname = result.stdout.split(dir+"/")[-1].split("\n")[0]
        return fname

    except:
	    print("Error")

filename = get_filename_from_aria2(DOWNLOAD_URL)

print(f"Detected Filename: {filename}")

c = input("Want to change filename? (y/n): ")

if c == "y":
    filename = input("Enter new filename: ")

print("Downloading:", filename)

!aria2c -x 16 -s 16 -k 1M -o "$filename" "$DOWNLOAD_URL"

print("Download complete.")

## Rclone (Faster)

In [ ]:
from google.colab import files
print("Upload rclone.conf file: ", end="")
uploaded = files.upload()

print("\nRclone Remotes: ")
!rclone --config=/content/rclone.conf listremotes

In [ ]:
# ========= EDIT REMOTE NAME ACCORDING TO RCLONE CONFIG ==========
remote_name = "remote_name" # Only the name, no colons
# ========= EDIT REMOTE NAME ACCORDING TO RCLONE CONFIG ==========

# cmd = [
#     "rclone", "copy", filename, f"{remote_name}:/Movies",
#     "--config=/content/rclone.conf",
#     "-v",
#     "--stats", "5s",
#     "--transfers", "4",
#     "--checkers", "4",
#     "--onedrive-chunk-size", "100M",
#     "--tpslimit", "10",
#     "--retries", "10"
# ]

cmd = [ 
    "rclone", "copy", filename, f"{remote_name}:/Movies", 
    "--config=/content/rclone.conf", 
    "-v", 
    "--stats", "10s", 
    "--transfers", "1", 
    "--checkers", "1", 
    "--onedrive-chunk-size", "200M", 
    "--multi-thread-streams", "8", 
    "--tpslimit", "10", 
    "--retries", "10",
    "--bind", "0.0.0.0"
]

# This runs the command and prints output line-by-line in real time
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)
for line in process.stdout:
    print(line, end='')
process.wait()

# Series Upload from Zip File

In [ ]:
import os
from urllib.parse import urlparse, unquote
import subprocess

DOWNLOAD_URL = input("Enter zip download URL: ")

def get_filename_from_aria2(url):
    try:
        result = subprocess.run(
            ['aria2c', '--dry-run', url],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            check=True
        )

        dir = os.getcwd().replace("\\", "/")
        fname = result.stdout.split(dir+"/")[-1].split("\n")[0]
        return fname

    except:
	    print("Error")

filename = get_filename_from_aria2(DOWNLOAD_URL)

print(f"Detected Filename: {filename}")

c = input("Want to change filename? (y/n): ")

if c == "y":
    filename = input("Enter new filename: ")

print("Downloading:", filename)

!aria2c -x 16 -s 16 -k 1M -o "$filename" "$DOWNLOAD_URL"

print("Download complete.")

In [ ]:
print(f"Unzipping {filename}")
!unzip "$filename"

In [ ]:
folderpath = input("Copy Extracted Folder Path and Paste Here: ")
sfolder = folderpath.split('/')[-1]

from google.colab import files
print("Upload rclone.conf file: ", end="")
uploaded = files.upload()

print("\nRclone Remotes: ")
!rclone --config=/content/rclone.conf listremotes

In [ ]:
# ========= EDIT REMOTE NAME ACCORDING TO RCLONE CONFIG ==========
remote_name = "remote_name" # Only the name, no colons
# ========= EDIT REMOTE NAME ACCORDING TO RCLONE CONFIG ==========

# cmd = [
#     "rclone", "copy", folderpath, f"{remote_name}:/Series/{sfolder}/",
#     "--config=/content/rclone.conf",
#     "-v",
#     "--stats", "5s",
#     "--transfers", "4",
#     "--checkers", "4",
#     "--onedrive-chunk-size", "100M",
#     "--tpslimit", "10",
#     "--retries", "10"
# ]

cmd = [ 
    "rclone", "copy", folderpath, f"{remote_name}:/Series/{sfolder}/",
    "--config=/content/rclone.conf", 
    "-v", 
    "--stats", "10s", 
    "--transfers", "4", 
    "--checkers", "4", 
    "--onedrive-chunk-size", "200M",
    "--tpslimit", "10", 
    "--retries", "10",
    "--bind", "0.0.0.0"
]

# This runs the command and prints output line-by-line in real time
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)
for line in process.stdout:
    print(line, end='')
process.wait()